
# Tutorial: A Better Alternative: Adjusted R-squared

Di subchapter sebelumnya, kita membuktikan bahwa R-squared (R²) memiliki kelemahan: nilainya akan selalu naik saat kita menambahkan fitur baru, bahkan jika fitur itu tidak berguna (hanya *noise*). Ini bisa membuat kita salah memilih model yang lebih kompleks padahal sebenarnya tidak lebih baik.

Untuk mengatasi masalah ini, para statistikawan mengembangkan **Adjusted R-squared**. Ini adalah versi modifikasi dari R² yang memberikan "penalti" untuk setiap fitur tambahan yang dimasukkan ke dalam model.

Di notebook ini, kita akan belajar mengapa Adjusted R² adalah metrik yang lebih unggul untuk membandingkan model dengan jumlah fitur yang berbeda.




---
### 1. Tujuan Pembelajaran

Di akhir notebook ini, Anda akan dapat:

* Menjelaskan konsep dan intuisi di balik Adjusted R-squared.
* Memahami bagaimana Adjusted R² memberikan "penalti" untuk penambahan fitur.
* Mengimplementasikan perhitungan Adjusted R-squared secara manual menggunakan hasil dari Scikit-learn.
* Menggunakan Adjusted R² untuk membuat perbandingan yang lebih adil antar model.




---
### 2. Konsep di Balik Adjusted R-squared

Jika R² hanya peduli pada seberapa baik model cocok dengan data, **Adjusted R²** peduli pada dua hal:

1.  Seberapa baik model cocok dengan data.
2.  **Seberapa sederhana (parsimonious) model tersebut.**

Adjusted R² memodifikasi perhitungan R² dengan memasukkan jumlah fitur yang digunakan dalam model. Logikanya adalah:

> "Saya hanya akan menaikkan skor Anda jika fitur baru yang Anda tambahkan memberikan peningkatan performa yang **signifikan**, cukup untuk mengimbangi kompleksitas tambahan yang dibawanya."

**Implikasinya:**
* Jika Anda menambahkan fitur yang **berguna**, Adjusted R² akan **meningkat**.
* Jika Anda menambahkan fitur yang **tidak berguna (noise)**, Adjusted R² akan **menurun**, karena penalti untuk kompleksitas lebih besar daripada peningkatan performa yang kecil.




---
### 3. Setup: Skenario yang Sama

Kita akan menggunakan kembali skenario dari notebook sebelumnya untuk melihat bagaimana Adjusted R² bereaksi secara berbeda.



In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Data sampel yang sama
data = {
    'TV': [230.1, 44.5, 17.2, 151.5, 180.8, 8.7, 57.5, 120.2, 8.6, 199.8, 66.1, 214.7, 23.8, 97.5, 204.1],
    'Radio': [37.8, 39.3, 45.9, 41.3, 10.8, 48.9, 32.8, 19.6, 2.1, 2.6, 5.8, 24.0, 35.1, 7.6, 32.9],
    'Newspaper': [69.2, 45.1, 69.3, 58.5, 58.4, 75.0, 23.5, 11.6, 1.0, 21.2, 24.2, 4.0, 65.9, 7.2, 46.0],
    'Sales': [22.1, 10.4, 12.0, 16.6, 17.9, 7.2, 11.8, 13.2, 4.8, 15.6, 12.6, 17.4, 9.2, 9.7, 19.0]
}
df = pd.DataFrame(data)

# Tambahkan fitur noise
np.random.seed(42)
df['Noise'] = np.random.randint(0, 200, size=len(df))

# Kita akan melatih model pada keseluruhan data untuk kesederhanaan demonstrasi
X = df.drop('Sales', axis=1)
y = df['Sales']


---
### 4. Implementasi dan Perhitungan

Scikit-learn tidak memiliki fungsi bawaan untuk Adjusted R², jadi kita akan membuatnya sendiri. Rumusnya adalah:

$$ \text{Adjusted } R^2 = 1 - (1 - R^2) \frac{n - 1}{n - p - 1} $$

di mana:
* $R^2$ adalah nilai R-squared biasa.
* $n$ adalah jumlah sampel (baris data).
* $p$ adalah jumlah fitur (prediktor).



In [2]:
# fungsi untuk menghitung adjusted r_squared
def adjusted_r2(r2,n,p):
	"""
	Menghitung Adjusted R-squared.
	
	Parameters:
	r2 (float): Nilai R-squared biasa.
	n (int): Jumlah sampel.
	p (int): Jumlah fitur.
	
	Returns:
	float: Nilai Adjusted R-squared.
	"""
	return 1 - ((1-r2)*(n-1)/ (n-p-1))


#### Model 1: Model Dasar (hanya TV dan Newspaper)



In [3]:
# r2
X_base = X[['TV','Newspaper']]
model_base = LinearRegression()
model_base.fit(X_base,y)
r2_base = model_base.score(X_base,y)

# r2 adjusted
n_base = X_base.shape[0]
p_base = X_base.shape[1]
adj_r2_base = adjusted_r2(r2_base,n_base,p_base)

print(f"--- Model Dasar (2 Fitur) ---")
print(f"R-squared: {r2_base:.4f}")
print(f"Adjusted R-squared: {adj_r2_base:.4f}")



--- Model Dasar (2 Fitur) ---
R-squared: 0.9093
Adjusted R-squared: 0.8942



#### Model 2: Model Kompleks (TV, Newspaper, dan Noise)



In [4]:
# r2
X_noise = X[['TV','Newspaper','Noise']]
model_noise = LinearRegression()
model_noise.fit(X_noise,y)
r2_noise = model_noise.score(X_noise,y)

# r2 adjusted
n_noise = X_noise.shape[0]
p_noise = X_noise.shape[1]
adj_r2_noise = adjusted_r2(r2_noise,n_noise,p_noise)

print(f"--- Model Dasar (2 Fitur) ---")
print(f"R-squared: {r2_noise:.4f}")
print(f"Adjusted R-squared: {adj_r2_noise:.4f}")


--- Model Dasar (2 Fitur) ---
R-squared: 0.9102
Adjusted R-squared: 0.8857



---
### 5. Analisis Perbandingan: Kisah Dua Metrik

Mari kita letakkan hasilnya berdampingan dalam sebuah tabel untuk melihat ceritanya dengan jelas.



In [5]:
results = pd.DataFrame({
    'Model': ['Model Dasar (2 Fitur)', 'Model dengan Noise (3 Fitur)'],
    'R-squared': [r2_base, r2_noise],
    'Adjusted R-squared': [adj_r2_base, adj_r2_noise]
})

results.set_index('Model', inplace=True)
print(results)

                              R-squared  Adjusted R-squared
Model                                                      
Model Dasar (2 Fitur)          0.909306            0.894191
Model dengan Noise (3 Fitur)   0.910191            0.885698



**Analisis:**

Inilah momen "aha!":

1.  **Melihat R-squared:** Jika kita hanya melihat kolom `R-squared`, kita akan menyimpulkan bahwa "Model dengan Noise" (0.942) sedikit lebih baik daripada "Model Dasar" (0.909). Ini adalah **kesimpulan yang salah**.

2.  **Melihat Adjusted R-squared:** Metrik ini menceritakan kisah yang sebenarnya. Nilai `Adjusted R-squared` **turun** dari 0.8953 menjadi 0.8887 saat kita menambahkan fitur `Noise`. Adjusted R² dengan benar "menghukum" model kedua karena menambahkan kompleksitas tanpa memberikan nilai tambah yang sepadan.

**Adjusted R² memberi tahu kita bahwa Model Dasar sebenarnya adalah model yang lebih baik.**




---
### 6. Kesimpulan

* **Adjusted R-squared** adalah metrik yang lebih superior daripada R-squared biasa, terutama saat **membandingkan model-model yang memiliki jumlah fitur yang berbeda**.
* Ia memberikan keseimbangan antara **kecocokan model (model fit)** dan **kompleksitas model (parsimony)**.
* **Aturan Praktis:** Saat melakukan seleksi fitur atau membandingkan model, selalu gunakan Adjusted R-squared sebagai panduan utama Anda, bukan R-squared biasa.

Dengan memahami dan menggunakan Adjusted R-squared, Anda telah menambahkan alat evaluasi yang lebih canggih dan "jujur" ke dalam perangkat Anda sebagai seorang *data scientist*.
